# 02 RAG | 01 Chat Completion Text | 02 SDK

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed ([Details and instructions can be found here](../01_CreateEnvironment/01_Environment.ipynb))

## Step 1: Create OpenAIClient / ChatClient

The `AzureOpenAIClient` from the Azure.AI.OpenAI NuGet package offers a unified interface to create various specialized clients, each designed to handle specific tasks.
In this notebook the `ChatClient` is used.



In [1]:
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.2"
#r "nuget: DotNetEnv, 3.1.1"

using Azure; 
using Azure.AI.OpenAI;
using OpenAI.Chat;
using DotNetEnv;
using System.IO;
using System.Text.Json; 
using System.ClientModel;

//configuration file is created during environment creation
static string configurationFile = @"../../Configuration/application.env";
Env.Load(configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("WS_AOAI_APIKEY") ?? "WS_AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("WS_AOAI_ENDPOINT") ?? "WS_AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("WS_CHATCOMPLETION_DEPLOYMENTNAME") ?? "WS_CHATCOMPLETION_DEPLOYMENTNAME not found";

ApiKeyCredential apiKeyCredential = new ApiKeyCredential(oAiApiKey);
AzureOpenAIClient azureOpenAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), apiKeyCredential);
ChatClient chatClient = azureOpenAIClient.GetChatClient(chatCompletionDeploymentName);

Console.WriteLine($"AzureOpenAI Client created...");
Console.WriteLine($"ChatClient created...");

Installed Packages Azure.AI.OpenAI, 2.2.0-beta.2 DotNetEnv, 3.1.1

AzureOpenAI Client created...
ChatClient created...


## Step 1: Basic Interaction

The following cell, demonstrate a basic interaction using the `chatClient` object. In this case, the system message is used to provide instructions or settings for the assistant, such as its personality or behavior.

The `ChatCompletionsOptions` object gives more fine granular control how the model should process the provided prompt.

System prompt: `"You are an AI assistance who helps finding information around international sport events"`

Prompt: `"Who won the super bowl 2024?"`

Because the GPT-4o model is trained with data up to October 2023 the model will answer with some information that the Super Bowl 2024 hasn't yet taken place.

In [2]:
string systemMessage = "You are an AI assistance who helps finding information around international sport events";
string userMessage = "Who won the super bowl 2024?";

ChatCompletionOptions chatCompletionOptions = new ChatCompletionOptions(){
    Temperature = 0.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.7f,
    PresencePenalty = 0.7f,
};
chatCompletionOptions.StopSequences.Add("\n");

ChatCompletion chatCompletion = await chatClient.CompleteChatAsync(
    messages: [
        new SystemChatMessage(systemMessage),
        new UserChatMessage(userMessage)
    ], 
    options: chatCompletionOptions
);

Console.WriteLine($"Model response: \n {chatCompletion.Content[0].Text}"); 


Model response: 
 As of my last update in October 2023, the Super Bowl for the year 2024 has not yet occurred. The event is typically held in February, so you'll need to check a reliable sports news source or the NFL's official website closer to that time for the most current information on the winner.


## Step 2: Complex interaction - Chat History

### Prepare chat history

Chat history is prepared with additional grounding information, information who won the Super Bowl in 2024 is added.

Chat History: 

```html
    - Who won the Super Bowl in 2024?
    - The Kansas City Chiefs won the Super Bowl in 2024!
    
```



In [3]:
string systemMessage = @"
    You are an AI assistance who helps finding information around international sport events.
    To answer questions you just take information provided to you!
    You don't use your training data.

    The Kansas City Chiefs won the Super Bowl 2024!
";
string prompt = "Who won the super bowl 2024?";

List<ChatMessage> chatMessages = new List<ChatMessage>();
chatMessages.Add(new SystemChatMessage(systemMessage));
chatMessages.Add(new UserChatMessage(prompt));

Console.WriteLine($"Chat messages created ...");


Chat messages created ...


### Model call

The chat history is provided to the model for completion

In [4]:
ChatCompletionOptions chatCompletionOptions = new ChatCompletionOptions(){
    Temperature = 0.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.7f,
    PresencePenalty = 0.7f,
};
chatCompletionOptions.StopSequences.Add("\n");

ChatCompletion chatCompletion = await chatClient.CompleteChatAsync(
    messages: chatMessages, 
    options: chatCompletionOptions
);

foreach (ChatMessageContentPart chatMessageContentPart in chatCompletion.Content){
    Console.WriteLine($"Model response: \n {chatMessageContentPart.Text}"); 
}

Model response: 
 The Kansas City Chiefs won the Super Bowl 2024!


## Step 3: Check additional response values

### Token Usage

Provide tokens consumed by the `GetChatCompletionsAsync()` call.

In [5]:
Console.WriteLine($"Input Tokens: {chatCompletion.Usage.InputTokenCount}");
Console.WriteLine($"Output Tokens: {chatCompletion.Usage.OutputTokenCount}");
Console.WriteLine($"Total Tokens:  {chatCompletion.Usage.TotalTokenCount}");

Input Tokens: 69
Output Tokens: 13
Total Tokens:  82


## Step 4: Streaming Response

`CompleteChatStreamingAsync()` provides results as stream as soon as they are provided by the model..

In [8]:
ChatCompletionOptions chatCompletionOptions = new ChatCompletionOptions(){
    Temperature = 0.0f,
    TopP = 1.0f,
    FrequencyPenalty = 0.7f,
    PresencePenalty = 0.7f,
};
chatCompletionOptions.StopSequences.Add("\n");

Console.WriteLine("Model response: ");
await foreach (var chatMessageContentPart in chatClient.CompleteChatStreamingAsync(
    messages: chatMessages, 
    options: chatCompletionOptions))
{
    ChatMessageContentPart? contentUpdate = chatMessageContentPart.ContentUpdate.FirstOrDefault();
    Console.Write($"{contentUpdate?.Text}"); 
}


Model response: 
The Kansas City Chiefs won the Super Bowl 2024!